## Welcome to the ivadomed's first tutorial: **One-class segmentation with 2D U-Net**

In this tutorial, we will be looking at how to train a two-dimensional (2D) segmentation model for segmenting the spinal cord with a single label on multiple Magnetic Resonance (MR) contrasts. The model will then be evaluted using various metrics like the Dice coefficient, Hausdorff distance, etc. This tutorial also provides visualizations of training curves and the segmented images on Tensorboard. 

⚠️ Before getting started, please ensure that you: 

1.   Are connected to the GPU. You can do this by doing the following from the task bar on the top: `Runtime` $\to$ `Change Runtime type` $\to$ `Hardware accelerator: GPU`
2.   **Are running this tutorial from _your_ Google Drive. You can do this by going to: `File` $\to$ `Save a Copy in Drive`.**


In [ ]:
# @title Fetch configuration file
# fetch the configuration (config) file to be used for this tutorial
!wget https://raw.githubusercontent.com/ivadomed/ivadomed/master/ivadomed/config/config.json ./content

In [ ]:
# @title Install Dependencies

!pip install imgaug==0.2.5 --quiet 
!pip install ivadomed --quiet 

%load_ext tensorboard

     |████████████████████████████████| 562 kB 11.6 MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 2.2 MB/s eta 0:08:52tcmalloc: large alloc 1147494400 bytes == 0x561b539ee000 @  0x7f497abaf615 0x561b1a7764cc 0x561b1a85647a 0x561b1a7792ed 0x561b1a86ae1d 0x561b1a7ece99 0x561b1a7e79ee 0x561b1a77abda 0x561b1a7ecd00 0x561b1a7e79ee 0x561b1a77abda 0x561b1a7e9737 0x561b1a86bc66 0x561b1a7e8daf 0x561b1a86bc66 0x561b1a7e8daf 0x561b1a86bc66 0x561b1a7e8daf 0x561b1a77b039 0x561b1a7be409 0x561b1a779c52 0x561b1a7ecc25 0x561b1a7e79ee 0x561b1a77abda 0x561b1a7e9737 0x561b1a7e79ee 0x561b1a77abda 0x561b1a7e8915 0x561b1a77aafa 0x561b1a7e8c0d 0x561b1a7e79ee
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:10:44tcmalloc: large alloc 1434370048 bytes == 0x561b98044000 @  0x7f497abaf615 0x561b1a7764cc 0x561b1a85647a 0x561b1a7792ed 0x561b1a86ae1d 0x561b1a7ece99 0x561b1a7e79ee 0x561b1a77abda 0x561b1a7ecd00 0x561b1a7e79ee

### Download the Dataset

We will be using a publicly available dataset consisting of the MRI data of the spinal cord. This dataset is a subset of the [spine-generic multi-center dataset](https://github.com/spine-generic/data-multi-subject) and has been pre-processed to facilitate training/testing of a new model. Namely, for each subject, all six contrasts were co-registered together. Semi-manual cord segmentation for all modalities and manual cerebrospinal fluid labels for T2w modality were created. More details can be found [here](https://github.com/ivadomed/ivadomed/blob/master/dev/prepare_data/README.md).

In addition to the MRI data, this sample dataset also includes a trained model for spinal cord segmentation. The size of the dataset is about 490MB. Please run the following cell to download the dataset.

In [ ]:
# @title Run Me to Download the Dataset!

# download the dataset
!ivadomed_download_data -d data_example_spinegeneric

2021-11-18 16:40:49.160 | INFO     | ivadomed.utils:init_ivadomed:408 - 
ivadomed (2.9.0)

Trying URL: https://github.com/ivadomed/data_example_spinegeneric/archive/r20200825.zip
Downloading: data_example_spinegeneric-r20200825.zip
Unzip data to: /tmp/tmpzfsxoabq
Removing temporary folders...
Folder Created: /content/data_example_spinegeneric
--2021-11-18 16:41:31--  https://raw.githubusercontent.com/ivadomed/ivadomed/master/ivadomed/config/config.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3699 (3.6K) [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]   3.61K  --.-KB/s    in 0s      

2021-11-18 16:41:31 (33.5 MB/s) - ‘config.json’ saved [3699/3699]

--2021-11-18 16:41:31--  http://./content
Resolving . (.)... fai

### Configuration File
In `ivadomed`, the training is orchestrated by a configuration file. In short, it is the JSON file that contains all the parameters used for loading the data, training and evaluating the model. An in-depth documentation on how to use the configuration file is available [here](https://ivadomed.org/configuration_file.html). Some examples of configuration files are available in the `ivadomed/config/` folder [here](https://github.com/ivadomed/ivadomed/tree/master/ivadomed/config).

In this tutorial, we will be using the configuration file: `ivadomed/config/config.json`. This is already downloaded for you and can be seen under Colab's `Files` (📁) tab on the left.

Open this file and follow on for more information on some of the key parameters for performing the one-class 2D segmentation:


1. `command` - The task to perform. This can either be "train" or "test". For training the model, we first set this key to "train".
```json
    "command": "train"
```

2. `path_output` - The name of the folder that will be populated by the output files (e.g. the trained model, predictions, results, etc.)
```json
    "path_output": "spineGeneric"
```

3. `loader_parameters:path_data` - The location of the dataset. As discussed in [Data](https://ivadomed.org/data.html), the dataset must conform to the BIDS standard. This value can be modified so as to point to the correct location of the downloaded dataset.
```json
    path_data: "data_example_spinegeneric"
```

4. `loader_parameters:target_suffix` - The suffix for the name of the ground truth (GT) segmentation file. The GT is located under the `DATASET/derivatives/labels` folder. For this tutorial, the suffix is `_seg-manual`.
```json
    "target_suffix": ["_seg_manual"]
```

5. `loader_parameters:contrast_params` - A dicitionary containing the contrasts of interest.
```json
    "contrast params": {
        "training_validation": ["T1w", "T2w", "T2star"],
        "testing": ["T1w", "T2w", "T2star"],
        "balance": {}
    }
```

6. `loader_parameters:slice_axis` - The orientation of the 2D slice to use with the model.
```json
    "slice_axis": "axial"
```

7. `loader_parameters:multichannel` - Turn on/off multi-channel training. If true, each sample has several channels, where each channel is an image contrast. If false, only one image contrast is used per sample.
```json
    "multichannel": false
```

⚠️ **Note**: The multichannel approach requires that for each subject, the image contrasts are co-registered. This implies that a ground truth segmentation is aligned with all contrasts, for a given subject. In this tutorial, only a single channel will be used.

8. `training_time:num_epochs` - The maximum number of epochs that will be run during training. Each epoch is composed of a training part and an validation part. It should be a positive integer.
```json
    "num_epochs": 100
```

### Modify the Config File

Now that we know how the config file is structured, open the `config.json` file under the "Files" tab on the left. This should let you edit the contents of the json file as mentioned above. Change the following parameters:

1. `"path_output": "spineGeneric_gpu"`  (just to differentiate the results obtained from the GPU)
2. `"debugging": true`     (to visualize training on Tensorboard)
3. `"num_epochs": 20`      (running on a few epochs for the purpose of this tutorial)

### Train the Model

Once the config file is saved and ready, the following command is used for training:
```shell
    ivadomed --train -c config.json --path-data path/to/bids/data --path-output path/to/output/directory
```

If the `--path_data` and `--path_output` keys are already mentioned in the config file then they do not need to be specified again. The **shorter command** shown below can be run instead: 
```shell
    ivadomed --train -c config.json
```

⚠️ **Note**: If a compatible GPU is available, it will be used by default (see the `"gpu_id"` key in the config file). Otherwise, training will use the CPU, which will take a prohibitively long computational time (several hours).

The main parameters of the training scheme and model will be displayed on the terminal, followed by the loss value on training and validation sets at every epoch. To know more about the meaning of each parameter, go to the [Configuration File](https://ivadomed.org/configuration_file.html). The value of the loss should decrease during the training.

After 20 epochs (see "num_epochs" in the configuration file), the Dice score on the validation set should be ~0.9.

In [ ]:
# @title Run me to start the training!

# train the model
!ivadomed --train -c config.json

2021-11-18 16:48:46.573 | INFO     | ivadomed.utils:init_ivadomed:408 - 
ivadomed (2.9.0)

2021-11-18 16:48:46.576 | INFO     | ivadomed.config_manager:_display_differing_keys:150 - Adding the following keys to the configuration file
2021-11-18 16:48:46.576 | INFO     | ivadomed.config_manager:deep_dict_compare:43 -     log_file: log
2021-11-18 16:48:46.576 | INFO     | ivadomed.config_manager:deep_dict_compare:43 -     loader_parameters: is_input_dropout: False
2021-11-18 16:48:46.576 | INFO     | ivadomed.config_manager:deep_dict_compare:43 -     default_model: is_2d: True
2021-11-18 16:48:46.576 | INFO     | ivadomed.config_manager:_display_differing_keys:152 - 

2021-11-18 16:48:46.576 | INFO     | ivadomed.utils:get_path_output:371 - CLI flag --path-output not used to specify output directory. Will check config file for directory...
2021-11-18 16:48:46.577 | INFO     | ivadomed.utils:get_path_data:383 - CLI flag --path-data not used to specify BIDS data directory. Will check confi

In [ ]:
# @title Visualize loss curves on Tensorboard

# see the training progress on Tensorboard
# note that the output folder is the same "path_output" folder used in the config file
%tensorboard --logdir spineGeneric_gpu

<IPython.core.display.Javascript object>

### Evaluate the Model

To test the trained model on the testing subset of the dataset and compute the evaluation metrics, run the following command: 
```shell
    ivadomed --test -c config.json --path-data path/to/bids/data --path-output path/to/output/directory
```

Again, if `--path_data` and `--path_output` are already mentioned in the config file, use the command below instead:
```shell
    ivadomed --test -c config.json
```

The model’s parameters will be displayed in the cell's output, followed by a preview of the results for each image. The resulting segmentation is saved for each image in the `<PATH_TO_OUT_DIR>/pred_masks` while a csv file, saved in `<PATH_TO_OUT_DIR>/results_eval/evaluation_3Dmetrics.csv`, contains all the evaluation metrics. For more details on the evaluation metrics, see `ivadomed.metrics` [here](https://ivadomed.org/api_ref.html#module-ivadomed.metrics).

In [ ]:
# @title Run me to test the model!

# test the model
!ivadomed --test -c config.json

2021-11-18 16:55:56.370 | INFO     | ivadomed.utils:init_ivadomed:408 - 
ivadomed (2.9.0)

2021-11-18 16:55:56.373 | INFO     | ivadomed.config_manager:_display_differing_keys:150 - Adding the following keys to the configuration file
2021-11-18 16:55:56.373 | INFO     | ivadomed.config_manager:deep_dict_compare:43 -     log_file: log
2021-11-18 16:55:56.373 | INFO     | ivadomed.config_manager:deep_dict_compare:43 -     loader_parameters: is_input_dropout: False
2021-11-18 16:55:56.374 | INFO     | ivadomed.config_manager:deep_dict_compare:43 -     default_model: is_2d: True
2021-11-18 16:55:56.374 | INFO     | ivadomed.config_manager:_display_differing_keys:152 - 

2021-11-18 16:55:56.374 | INFO     | ivadomed.utils:get_path_output:371 - CLI flag --path-output not used to specify output directory. Will check config file for directory...
2021-11-18 16:55:56.374 | INFO     | ivadomed.utils:get_path_data:383 - CLI flag --path-data not used to specify BIDS data directory. Will check confi

In [ ]:
# @title Save the results in a zip file!
# @markdown Now that training and testing are done, we would like to download
# @markdown the results locally for further anaylsis. For that, we first
# @markdown create a `.zip` file of the results folder and then download
# @markdown the zipped file manually. 

# first, zip the results folder
!zip -r --quiet spineGeneric_gpu.zip spineGeneric_gpu/
print("Zip file created!")

Zip file created!


Now, check out the Files tab on the right. You can find spineGeneric_gpu.zip when you refresh the content (see the top bar) and then download the zip file to your browser's standard Downloads folder.

The test image segmentations are stored in `<PATH_TO_OUT_DIR>/pred_masks/` and have the same name as the input image with the suffix `_pred`. To visualize the segmentation of a given subject, you can use any Nifti image viewer (e.g. [ITK-SNAP](http://www.itksnap.org/pmwiki/pmwiki.php), [FSLeyes](https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/userdoc/)). 

After the training for 100 epochs, the segmentations should be similar to the one presented in the following image. The output and ground truth segmentations of the spinal cord are presented in red (subject `sub-mpicbs06` with contrast T2w):

<img src="https://raw.githubusercontent.com/ivadomed/doc-figures/main/tutorials/one_class_segmentation_2d_unet/sc_prediction.png">

⚠️ **Note**: In case you prefer running things on the terminal instead of notebooks, ivadomed also makes that possible. Head over to [this](https://ivadomed.org/tutorials/one_class_segmentation_2d_unet.html) page that explains this tutorial from the terminal. However, before doing that please ensure that you have installed `ivadomed` locally. 

So, that was it for the first tutorial! We saw a simple example of how `ivadomed` can be used to segment the spinal cord. Please try the other tutorials to get a better feel of what `ivadomed` has to offer. 